In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np


test_data_k = np.load(f'./experiment4/test1_K_test.npy')

test_data_p = np.load(f'./experiment4/test3_P_test.npy')

test_data_Mg = np.load(f'./experiment4/test7_Mg_test.npy')

In [3]:
from creators.CNN3DCreator import  CNN3DNetworkParams
from creators.CRNNCreator import CRNNNetworkParams
from creators.ModelParams import Conv1DParams, Conv2DParams, Conv3DParams, DeepLayerParams, FlattenParams, GenericNeuralNetworkParams, RandomForestParams, RecurrentLayerParams, SVRParams
from creators.CNN1DCreator import  CNN1DNetworkParams
from creators.CNN2DCreator import  CNN2DNetworkParams
from prep.HelperBlocks import pipe, split_3d_to_timeseries
import tensorflow as tf
from prep.MSCBlock import msc_block, msc_block1d

from prep.Normlize import normalize_layers

drop=0.53
epochs=50
optimizer = tf.keras.optimizers.Adam
folds = 5

get_params_cnn2d_task1 = lambda: CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(4,4),pool_size=(3,3),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(4,4),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

get_params_cnn2d_task2 = lambda: CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(3,3),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(5,5),pool_size=(3,3),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=1024, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer,loss=tf.losses.Huber()
        )
)

c:\ProgramData\Anaconda3\envs\thesis\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [5]:
predicted_Mg = []
for i in range(5):
    params = get_params_cnn2d_task2()
    flow = params.generate_flow("model")
    flow.import_weights(test_data_Mg.shape, f'./best/Mg/{i}.h5')
    predicted_Mg.append(flow.predict(test_data_Mg))

predicted_Mg = np.array(predicted_Mg).mean(axis=0)
predicted_Mg.shape

37/37 [==============================] - 1s 16ms/step


(1154,)

In [6]:
predicted_K = []
for i in range(5):
    params = get_params_cnn2d_task1()
    flow = params.generate_flow("model")
    flow.import_weights(test_data_k.shape, f'./best/k/{i}.h5')
    predicted_K.append(flow.predict(test_data_k))

predicted_K = np.array(predicted_K).mean(axis=0)
predicted_K.shape

37/37 [==============================] - 1s 14ms/step


(1154,)

In [7]:
predicted_P = []
for i in range(5):
    params = get_params_cnn2d_task2()
    flow = params.generate_flow("model")
    flow.import_weights(test_data_p.shape, f'./best/p/{i}.h5')
    predicted_P.append(flow.predict(test_data_p))

predicted_P = np.array(predicted_P).mean(axis=0)
predicted_P.shape

37/37 [==============================] - 1s 15ms/step


(1154,)

In [8]:
import numpy as np


ph = np.load("./predict/ph.npy")
ph.shape

(1154,)

In [9]:
import pandas as pd


results = pd.DataFrame(columns=["P", "K", "Mg", "pH"])
results["P"] = predicted_P
results["K"] = predicted_K
results["Mg"] = predicted_Mg
results["pH"] = ph

In [10]:
results["sample_index"] = range(0, 1154)
results.set_index("sample_index").to_csv("submission.csv")